# Getting Courses
---

Set username and password if you want to login automatically. Not required.

In [1]:
USERNAME = ""
PASSWORD = ""

Log into temple and wait for the request session to be created.

In [ ]:
from temple.planner import TUPlanner

planner = TUPlanner(username=USERNAME, password=PASSWORD)

Once the session has been created, execute the cell below to get the code for the term you are planning course for.

In [ ]:
planner.print_terms(3)

Once you have retrieved the term code, update the `TERM_CODE` variable with the code and specify which courses you are planning to take.

In [7]:
TERM_CODE = 202503

# Plotting Courses
---

In [8]:
SELECTED_COURSES = ["MATH2043", "IH0851", "MATH2101", "CIS1068"] # "CIS2033"
# SELECTED_COURSES = ["CIS1057", "CIS1166", "ENG0802", "CIS1001", "SCTC1001"]

In [12]:
from temple.schedule import Schedule, ScheduleCompare
from itertools import product

def get_schedule_combos(_course_list: list[str]) -> list[tuple]:
    all_sections = []
    for course_name in set(_course_list):
        # Fetch all section info for selected courses
        success, result = planner.get_course_sections(course_name, TERM_CODE)

        if not success:
            print("Failed to get course info.")
            return []
        else:
            # Append list of sections for each course so that we can get the cartesian product
            all_sections.append([section for section in result])

    # Get the cartesian product of all sections
    return list(product(*all_sections))

In [13]:
schedule_combos = get_schedule_combos(SELECTED_COURSES)

In [ ]:
schedules = set(Schedule(schedule) for schedule in schedule_combos)
schedules = sorted(filter(ScheduleCompare.not_overlap, schedules), key=lambda x: ScheduleCompare.between_total(x) * ScheduleCompare.week_total(x))
schedules = list(schedules)[:5]
print(len(schedules))

In [ ]:
from IPython.display import Markdown, display


def render_table(headers: list[str], rows: list[list[str]]):
    # Create the header row
    md = "| " + " | ".join(headers) + " |\n"
    md += "| " + " | ".join(["---"] * len(headers)) + " |\n"  # Separator row

    # Add each row of data
    for row in rows:
        md += "| " + " | ".join(row) + " |\n"

    display(Markdown(md))

for schedule in schedules:
    print(
        f"WEEK_RANGE: {ScheduleCompare.week_range(schedule):.2f} hrs, "
        f"WEEK_TOTAL: {ScheduleCompare.week_total(schedule):.2f} hrs, "
        f"BREAK_TOTAL: {ScheduleCompare.between_total(schedule):.2f} hrs"
    )
    render_table(["Class", "Section"], [*schedule.get_classes().items()])

    schedule.show()

In [ ]:
import base64

def base64decode(_str: str):
    return base64.b64decode(_str.encode()).decode()

def base64encode(_str: str):
    return base64.b64encode(_str.encode()).decode()

print(base64decode("YXJyYXljb25uZWN0aW9uOjc="))

print(base64encode("test:test"))
print(base64encode(f"arrayconnection:{15}"))

In [ ]:
import requests
import json

with open("teacher_query.graphql", "r") as file:
    TEACHER_GQL = file.read()

with open("school_query.graphql", "r") as file:
    SCHOOL_GQL = file.read()

response = requests.post(
    "https://www.ratemyprofessors.com/graphql",
    json={
        "query": TEACHER_GQL,
        "variables": {
            "count": 20,
            "cursor": base64encode(f"arrayconnection:{-1}"),
            "query": {
                "text": "",
                "schoolID": base64encode("School-999"),
                "fallback": "true",
            },
        },
    },
    # json={
    #     "query": SCHOOL_GQL,
    #     "variables": {
    #         "count": 100,
    #         "cursor": base64encode(f"arrayconnection:{-1}"),
    #         "query": {
    #             "text": "Temple University",
    #         },
    #     },
    # },
    headers={"Authorization": f"Basic {base64encode("test:test")}"},
)


print(response.status_code)

try:
    teachers = [
        f"{teacher["node"]["firstName"]} {teacher["node"]["lastName"]}"
        for teacher in response.json()["data"]["search"]["teachers"]["edges"]
    ]

    print(json.dumps(teachers, indent=4))
    print(len(teachers))

    # schools = [
    #     school["node"]["name"]
    #     for school in response.json()["data"]["search"]["schools"]["edges"]
    # ]
    # print(json.dumps(schools, indent=4))
    # print(len(schools))
    # print(json.dumps(response.json(), indent=4))
except Exception:
    print(json.dumps(response.json(), indent=4))